```
[Title] Multivariate LSTM
[Author] Jason Brownlee
[Etidor] Yibeck.Lee(yibeck.lee@gmail.com)
[reference]
 - https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
```

In [78]:
# multivariate lstm example
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [79]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


In [80]:

# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
dataset


array([[ 10,  15,  25],
       [ 20,  25,  45],
       [ 30,  35,  65],
       [ 40,  45,  85],
       [ 50,  55, 105],
       [ 60,  65, 125],
       [ 70,  75, 145],
       [ 80,  85, 165],
       [ 90,  95, 185]])

In [81]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
 

In [82]:
print(X.shape, y.shape)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

(7, 3, 2) (7,)
[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


In [83]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')


In [84]:
# fit model
model.fit(X, y, epochs=200, verbose=1)


Epoch 1/200
7/7 [==============================] - 2s 315ms/step - loss: 19434.5723
Epoch 2/200
7/7 [==============================] - 0s 2ms/step - loss: 19061.4160
Epoch 3/200
7/7 [==============================] - 0s 1ms/step - loss: 18752.7715
Epoch 4/200
7/7 [==============================] - 0s 2ms/step - loss: 18406.1699
Epoch 5/200
7/7 [==============================] - 0s 2ms/step - loss: 18072.6621
Epoch 6/200
7/7 [==============================] - 0s 3ms/step - loss: 17749.9004
Epoch 7/200
7/7 [==============================] - 0s 714us/step - loss: 17431.2383
Epoch 8/200
7/7 [==============================] - 0s 2ms/step - loss: 17133.0723
Epoch 9/200
7/7 [==============================] - 0s 2ms/step - loss: 16882.9004
Epoch 10/200
7/7 [==============================] - 0s 2ms/step - loss: 16624.3848
Epoch 11/200
7/7 [==============================] - 0s 2ms/step - loss: 16382.9121
Epoch 12/200
7/7 [==============================] - 0s 2ms/step - loss: 16140.6387
Epoch 13/

In [85]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=1)


1/1 [==============================] - 1s 538ms/step


In [86]:
yhat

array([[208.16771]], dtype=float32)